In [66]:
import numpy as np
import pandas as pd
import scipy as sp
from linearmodels import PanelOLS
import statsmodels as sm
import statsmodels.formula.api as smf
import statsmodels.api as sm_api

In [92]:
df = pd.read_stata("AEJ2018_child_mortality.dta") 
df.count()
df = df[df.datedeath_cmc<1333]
df.count()

branchid              221
villageid             221
treatment             221
hhid                  221
childID               221
female                221
datebirth_m           221
datebirth_yr          221
datebirth_cmc         221
died                  221
datedeath_m           221
datedeath_yr          221
datedeath_cmc         221
aad                   221
dateofinterview_m     221
dateofinterview_yr    221
dtype: int64

In [93]:
df['death_u5'] = np.where((df['died']==1) & (df['aad']<60), 1, 0) 
df['death_u1'] = np.where((df['died']==1) & (df['aad']<12), 1, 0)
df['death_u1m'] = np.where((df['died'] == 1) & (df['aad']<1), 1,0)

In [95]:
df = df.groupby(['branchid', 'villageid', 'treatment'], as_index = False)[['death_u5', 
'death_u1','death_u1m']].sum().reset_index()
df.count() #in Stata collapse (sum) reduces # of obs from 11342 to 214, here after groupby only 123 obs left (?) 

index        123
branchid     123
villageid    123
treatment    123
death_u5     123
death_u1     123
death_u1m    123
dtype: int64

In [70]:
df.describe()

,branchid,villageid,treatment,hhid,childID,female,datebirth_m,datebirth_yr,datebirth_cmc,died,datedeath_m,datedeath_yr,datedeath_cmc,aad,dateofinterview_m,dateofinterview_yr,death_u5,death_u1,death_u1m
count,123.000000,123.000000,123.000000,123.000000,123.000000,123.000000,123.000000,123.000000,123.000000,123.000000,123.000000,123.000000,123.000000,123.000000,123.000000,123.00000,123.000000,123.000000,123.000000
mean,15.707317,1414.886179,0.512195,252806.040650,2.000000,0.764228,13.056911,3609.869873,2365.642334,1.796748,13.504066,3610.325195,2371.552734,5.910569,17.894308,3616.85376,1.796748,1.601626,0.975610
std,14.987827,238.781957,0.501896,146474.818855,1.361051,0.769088,9.427289,2073.143555,1357.780273,1.031943,9.693140,2073.362061,1360.626465,7.986681,10.371629,2077.30249,1.031943,1.022211,0.909543
min,1.000000,1001.000000,0.000000,100115.000000,1.000000,0.000000,1.000000,2008.000000,1304.000000,1.000000,1.000000,2008.000000,1304.000000,0.000000,9.000000,2013.00000,1.000000,0.000000,0.000000
25%,5.000000,1306.500000,0.000000,141278.000000,1.000000,0.000000,7.000000,2009.000000,1316.000000,1.000000,7.500000,2009.000000,1320.000000,0.000000,10.000000,2013.00000,1.000000,1.000000,0.000000
50%,5.000000,1347.000000,1.000000,202269.000000,2.000000,1.000000,11.000000,4016.000000,2611.000000,2.000000,11.000000,4016.000000,2616.000000,2.000000,18.000000,4026.00000,2.000000,1.000000,1.000000
75%,31.000000,1629.000000,1.000000,327338.500000,2.000000,1.000000,17.000000,4019.000000,2645.500000,2.000000,16.500000,4020.000000,2653.500000,9.000000,20.000000,4026.00000,2.000000,2.000000,1.500000
max,42.000000,1906.000000,1.000000,811137.000000,8.000000,3.000000,66.000000,12054.000000,7914.000000,6.000000,66.000000,12055.000000,7926.000000,39.000000,66.000000,12078.00000,6.000000,6.000000,4.000000


In [71]:
print(df.columns)
df = df.set_index(['branchid','villageid'])

Index(['branchid', 'villageid', 'treatment', 'hhid', 'childID', 'female',
       'datebirth_m', 'datebirth_yr', 'datebirth_cmc', 'died', 'datedeath_m',
       'datedeath_yr', 'datedeath_cmc', 'aad', 'dateofinterview_m',
       'dateofinterview_yr', 'death_u5', 'death_u1', 'death_u1m'],
      dtype='object')


In [72]:
from linearmodels import PanelOLS
model = PanelOLS(df['death_u5'],df['treatment'], entity_effects = True)
res = model.fit(cov_type='robust', cluster_entity=True)
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:               death_u5   R-squared:                        0.0014
Estimator:                   PanelOLS   R-squared (Between):             -0.0442
No. Observations:                 123   R-squared (Within):               0.0014
Date:                Mon, May 04 2020   R-squared (Overall):             -0.0341
Time:                        17:35:13   Log-likelihood                   -171.98
Cov. Estimator:                Robust                                           
                                        F-statistic:                      0.1583
Entities:                          12   P-value                           0.6915
Avg Obs:                       10.250   Distribution:                   F(1,110)
Min Obs:                       3.0000                                           
Max Obs:                       37.000   F-statistic (robust):             0.1716
                            

In [73]:
model = PanelOLS(df['death_u1'], df['treatment'], entity_effects = True)
res = model.fit(cov_type = 'robust', cluster_entity = True)
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:               death_u1   R-squared:                        0.0024
Estimator:                   PanelOLS   R-squared (Between):              0.0613
No. Observations:                 123   R-squared (Within):               0.0024
Date:                Mon, May 04 2020   R-squared (Overall):              0.0466
Time:                        17:35:22   Log-likelihood                   -171.47
Cov. Estimator:                Robust                                           
                                        F-statistic:                      0.2695
Entities:                          12   P-value                           0.6047
Avg Obs:                       10.250   Distribution:                   F(1,110)
Min Obs:                       3.0000                                           
Max Obs:                       37.000   F-statistic (robust):             0.2936
                            

In [75]:
model = PanelOLS(df['death_u1m'], df['treatment'], entity_effects = True)
res = model.fit(cov_type = 'robust', cluster_entity = True)
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:              death_u1m   R-squared:                        0.0079
Estimator:                   PanelOLS   R-squared (Between):              0.1643
No. Observations:                 123   R-squared (Within):               0.0079
Date:                Mon, May 04 2020   R-squared (Overall):              0.0916
Time:                        17:36:51   Log-likelihood                   -153.75
Cov. Estimator:                Robust                                           
                                        F-statistic:                      0.8810
Entities:                          12   P-value                           0.3500
Avg Obs:                       10.250   Distribution:                   F(1,110)
Min Obs:                       3.0000                                           
Max Obs:                       37.000   F-statistic (robust):             0.9166
                            

In [77]:
data_2 = pd.read_stata("AEJ2018_child_mortality_computation.dta") 
data_2.describe()
data_2.count()

branchid              11342
villageid             11342
treatment             11342
hhid                  11342
childID               11342
female                11342
datebirth_m           11342
datebirth_yr          11342
datebirth_cmc         11342
died                  11342
datedeath_m             390
datedeath_yr            389
datedeath_cmc           389
aad                     389
dateofinterview_m     11342
dateofinterview_yr    11342
count_month_u5        11342
death_under5          11342
date12m_cmc           11342
aabeginning           11342
aaend                 11342
count_month_u1         8808
death_under1          11342
count_month_u1m        6499
death_under1m          6499
dtype: int64

In [90]:
data_2 = data_2.groupby(['villageid', 'branchid', 'treatment'], as_index = False)[['death_under5','count_month_u5',
'death_under1', 'count_month_u1','death_under1m','count_month_u1m']].sum().reset_index()
data_2.count()


index              214
villageid          214
branchid           214
treatment          214
death_under5       214
count_month_u5     214
death_under1       214
count_month_u1     214
death_under1m      214
count_month_u1m    214
dtype: int64